In [ ]:
import pickle
import networkx as nx
import datetime
import pandas as pd 
import numpy as np
from tqdm import tqdm

# HF patients 

In [ ]:
with open('/projects/RALES TRIAL/1A/notspiro dataextraction/notspiro data/hf_patients.pickle', 'rb') as f:
    hf_patients = pickle.load(f)

In [ ]:
hf_ids = list(hf_patients.keys())
len(hf_ids)

# Identifying HF patients with dyspnea

In [ ]:
def graph_from_onto(onto):
    G = nx.DiGraph()
    cl_edges = []
    for s, ts in onto.items():
        for t in ts:
            cl_edges.append((s.replace('S-', ''), t.replace('S-', '')))
    G.add_edges_from(cl_edges)
    return G

def load_onto(o):
    with open(f'/projects/data/GS/{o}', 'rb') as f:
        onto = pickle.load(f)
    G = graph_from_onto(onto)
    return G

def expand_codes(onto, codes):
    expanded = {}
    for name, top_codes in codes.items():
        expanded[name] = set()
        for c in top_codes:
            expanded[name].add(c)
            if c in onto:
                expanded[name].update(nx.ancestors(onto, c))
            else:
                print("NOT FOUND:", c, name)
        print(name, len(top_codes), len(expanded[name]))
    return expanded

In [ ]:
dyspnea = { 
    'breathless_moderate_strenous_exertion' : ['161939006', '390871002', '390870001'],
    'breathless_mild_exertion':['161940008'],
    'chronic_dyspnea' : ['870535009'],
    'tired_on_least_exertion' : ['248269005'],
    'pain_provoked_by_exertion' : ['427341007'],
    'dyspnea_on_exertion' : ['60845006'],
    'dyspnea_at_rest' : ['161941007'],
    'chest_pain_on_exertion': ['81953000'],
    'chest_pain_at_rest': ['9267009'],    
}

In [ ]:
onto = load_onto('isa_rela_ch2pt_202009.pickle')
dyspnea = expand_codes(onto,dyspnea)

In [ ]:
with open('/projects/data/GS/pt2cui_pos_dates.pickle', 'rb') as f:
    pt2cui_pos_dates = pickle.load(f)

In [ ]:
def dates_dyspnea(pt_data, codes):
    dates = None
    for x in codes:
        if x in pt_data and len(pt_data[x])>=2:
            if dates == None:
                dates = pt_data[x]
            else: 
                dates = dates.union(pt_data[x])
    return dates

In [ ]:
rows = []
for pt in hf_ids:
    pt_data = pt2cui_pos_dates.get(pt, {})
    row = {'client_idcode': pt, 'start_date': hf_patients[pt]} 
    for concept, codes in dyspnea.items():
        dates = dates_dyspnea(pt_data, codes)
        if dates == None:
            row[f"{concept}_date"] = np.nan
        else:
            row[f"{concept}_date"] = dates
    rows.append(row)

In [ ]:
df = pd.DataFrame(rows)
df.shape

In [ ]:
df.head()

In [ ]:
%%time
df = df[['client_idcode']].join((df[i].explode() for i in df.iloc[:,1:]))

In [ ]:
%%time
for k in dyspnea.keys():
    df[f'{k}_date'] = pd.to_datetime(df[f'{k}_date']).dt.date

In [ ]:
%%time
for k in dyspnea.keys():
    df[f'{k}_delta'] = pd.to_datetime(df['start_date']) - pd.to_datetime(df[f'{k}_date'])

In [ ]:
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
for k in dyspnea.keys():
    t1 = df[f'{k}_delta'] >= min_time 
    t2 = df[f'{k}_delta'] <= max_time
    in_window = t1 & t2 
    df[f'{k}_in_window'] = in_window

In [ ]:
for k in dyspnea.keys():
    df[f'{k}_true_count'] = (
        df[f'{k}_in_window'].eq(True)
            .groupby(df['client_idcode']).transform('sum')
    )

In [ ]:
df = df.drop_duplicates('client_idcode') 
df.reset_index(drop=True, inplace=True)

# Preparing the data for extraction

In [ ]:
dyspnea_on_exertion = set()
for i,row in tqdm(df.iterrows()):
    if df.loc[i,'dyspnea_on_exertion_true_count'] >= 1:
        dyspnea_on_exertion.add(df.loc[i,'client_idcode'])
        
len(dyspnea_on_exertion)

In [ ]:
pain_provoked_by_exertion  = set()
for i,row in tqdm(df.iterrows()):
    if df.loc[i,'pain_provoked_by_exertion_true_count'] >= 1:
        pain_provoked_by_exertion.add(df.loc[i,'client_idcode'])
        
len(pain_provoked_by_exertion)

In [ ]:
dyspnea_at_rest = set()
for i,row in tqdm(df.iterrows()):
    if df.loc[i,'dyspnea_at_rest_true_count'] >= 1:
        dyspnea_at_rest.add(df.loc[i,'client_idcode'])
        
len(dyspnea_at_rest)

In [ ]:
chest_pain_on_exertion = set()
for i,row in tqdm(df.iterrows()):
    if df.loc[i,'chest_pain_on_exertion_true_count'] >= 1:
        chest_pain_on_exertion.add(df.loc[i,'client_idcode'])
        
len(chest_pain_on_exertion)

In [ ]:
chest_pain_at_rest = set()
for i,row in tqdm(df.iterrows()):
    if df.loc[i,'chest_pain_at_rest_true_count'] >= 1:
        chest_pain_at_rest.add(df.loc[i,'client_idcode'])
        
len(chest_pain_at_rest)

In [ ]:
tired_on_least_exertion = set()
for i,row in tqdm(df.iterrows()):
    if df.loc[i,'tired_on_least_exertion_true_count'] >= 1:
        tired_on_least_exertion.add(df.loc[i,'client_idcode'])
        
len(tired_on_least_exertion)

In [ ]:
dyspnea = dyspnea_on_exertion.union(pain_provoked_by_exertion,dyspnea_at_rest,chest_pain_on_exertion,chest_pain_at_rest,tired_on_least_exertion)
len(dyspnea)